In [1399]:
import pandas as pd
import numpy as np
import json

In [1400]:
df = pd.read_csv("shop_users.csv")

df_copy = df.copy()

columns = ['Unnamed: 0', 'common_count', 'is_closed', 'home_town', 'relation']

In [1401]:
df_copy.drop(columns=columns, inplace=True)

In [1402]:
df_copy.drop_duplicates(inplace=True)

In [1403]:
df_copy.dropna(subset=["games", "costs"], inplace=True)

In [1404]:
df_copy.reset_index(inplace=True)

In [1405]:
def text_to_json(text):
  if pd.isna(text):
    result = 'Город не указан'
  else:
    result = json.loads(text.replace("'", '"'))['title']
  return result

In [1406]:
df_copy["city"] = df_copy["city"].apply(text_to_json)

In [1407]:
df_copy.drop(labels=[3334, 3335, 3336, 3337], axis=0, inplace=True)

In [1408]:
df_copy["country"] = df_copy["country"].apply(lambda x: eval(x)["title"])

In [1409]:
def text_to_json2(text):
    if pd.isna(text):
        result = np.nan
    else:
        result = json.loads(text.replace("'", '"'))["time"]
    return result
        

In [1410]:
df_copy["last_seen"] = df_copy["last_seen"].apply(text_to_json2)

In [1411]:
def costs_func(costs_currency):
    return costs_currency.replace('"', "").replace("{", "").replace("}", "").replace("'", "").split(",")[0].split()[1]

def currency_func(costs_currency):
    return costs_currency.replace('"', "").replace("{", "").replace("}", "").replace("'", "").split(",")[1].strip().split()[1]
    
    

In [1412]:
df_copy["costs_sum"] = df_copy.costs.apply(costs_func)
df_copy['currency'] = df_copy.costs.apply(currency_func)

In [1413]:
df_copy.drop(labels="costs", axis=1, inplace=True)

In [1414]:
df_copy.rename(columns={"costs_sum": "costs"}, inplace=True)

In [1415]:
def games_func(games):
    return [games["name"] for games in eval(games)]

In [1416]:
df_copy['games_clean'] = df_copy['games'].apply(games_func)

In [1417]:
df_copy.drop(columns="games", inplace=True)

In [1418]:
df_copy.rename(columns={"games_clean": "games"}, inplace=True)

In [1419]:
df_copy.bdate = pd.to_datetime(df_copy.bdate, format="%d.%m.%Y")

In [1420]:
df_copy.last_seen = pd.to_datetime(df_copy.last_seen, unit="s", origin="unix")

In [1421]:
df_copy.last_seen = df_copy.last_seen.dt.strftime('%d.%m.%Y')

In [1422]:
df_copy.followers_count = df_copy.followers_count.astype('Int64')

In [1423]:
df_copy.costs = pd.to_numeric(df_copy.costs, downcast='float')

In [1424]:
df_copy["age"] = pd.Timestamp.today().year - df_copy.bdate.apply(lambda x: x.year)

In [1425]:
df_copy.drop(columns="bdate", inplace=True)

In [1426]:
df_copy.currency.unique()

array(['RUR', 'USD', 'KZT', 'EUR'], dtype=object)

In [1427]:
rates = dict(RUR=1, USD=100, EUR=106.30, KZT=5.18)

In [1428]:
df_copy["costs_to_rur"] = df_copy["currency"].map(rates) * df_copy["costs"]

In [1429]:
df_copy.drop(columns=["costs", "currency"], inplace=True)

In [1430]:
sex_dict = {1: "женский", 2: "мужской", 0: "не указан"}
df_copy.sex = df_copy.sex.apply(lambda x: sex_dict[x])

In [1431]:
df_copy.city = df_copy.city.str.lower()

In [1432]:
df_copy.loc[df_copy.city == 'moscow', 'city'] = 'москва'
df_copy.loc[df_copy.city.str.contains('petersburg'), 'city'] = 'санкт-петербург'

In [1433]:
df_copy.country = df_copy.country.str.lower()

In [1434]:
columns = ['id', 'age', 'sex', 'city', 'country', 'followers_count', 'last_seen', 'costs_to_rur', 'games']
df_copy = df_copy[columns]

In [1435]:
df_games = df_copy.explode("games", ignore_index=True)

In [1436]:
df_games.to_csv("df_games.csv", index=False)